In [34]:
!pip install kaggle

import os
os.environ['KAGGLE_USERNAME'] = 'jhyeonlee'
os.environ['KAGGLE_KEY'] = '511ead758e41fd0cd1c3800a8d735dc9'

!kaggle competitions download -c titanic

train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')

In [36]:
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [37]:
test_set.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [38]:
train_set = train_set.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
test_set = test_set.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
combine = [train_set, test_set]

In [39]:
sex_mapping = {'female': 1, 'male': 0}
embarked_mapping = {'S': 0, 'C': 1, 'Q': 2}
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)
    dataset['Embarked'] = dataset['Embarked'].map(embarked_mapping)

train_set.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,0.0
1,1,1,1,38.0,1,0,71.2833,1.0
2,1,3,1,26.0,0,0,7.9250,0.0
3,1,1,1,35.0,1,0,53.1000,0.0
4,0,3,0,35.0,0,0,8.0500,0.0


In [42]:
from keras.utils import np_utils

# prepare train & test data
X_train = train_set.drop(["Survived"], axis=1)
y_train = train_set["Survived"]
X_test  = test_set

X_train = np.array(X_train).astype('float32')
y_train = np.array(y_train).astype('int32')
y_train = np_utils.to_categorical(y_train, 2)
X_test = np.array(X_test).astype('float32')

#모델

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, Adadelta

In [49]:
random_state=21
lr = 0.001
desired_accuracy = 0.99
dropout_r1 = 0.5
dropout_r2 = 0.5
batch_size=1
epochs=2000

In [45]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > desired_accuracy:
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True
callbacks = myCallback()

In [46]:
model = tf.keras.models.Sequential([Dense(256, activation='tanh'),
                                   Dropout(dropout_r1),
                                   Dense(256, activation='tanh'),
                                   Dropout(dropout_r2),
                                   Dense(2, activation='sigmoid')])

In [47]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr),
              metrics=['accuracy'])

In [50]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    #steps_per_epoch=10,
                    epochs=epochs,
                    validation_split=0.1,
                    #validation_data=(X_val, y_val),
                    verbose=1,
                    callbacks=[callbacks])

801/801 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.6155 - val_loss: nan - val_accuracy: 0.6222
Epoch 9/2000
801/801 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.6155 - val_loss: nan - val_accuracy: 0.6222
Epoch 10/2000
801/801 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.6155 - val_loss: nan - val_accuracy: 0.6222
Epoch 11/2000
517/801 [==================>...........] - ETA: 0s - loss: nan - accuracy: 0.5880

KeyboardInterrupt: ignored

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()